In [0]:
dbutils.fs.ls("/")

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS main;
CREATE SCHEMA IF NOT EXISTS main.raw;

CREATE VOLUME IF NOT EXISTS main.raw.sales;

In [0]:
dbutils.fs.ls("dbfs:/Volumes/main/raw/sales")

In [0]:
df_raw = spark.read.format("csv").option("header",'true').option("inferSchema",'true').load("dbfs:/Volumes/main/raw/sales")

In [0]:
df_raw.printSchema()

## Create Bronze Volume

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS main.bronze;
CREATE VOLUME IF NOT EXISTS main.bronze.sales;

In [0]:
from pyspark.sql.functions import current_timestamp, input_file_name, col

BRONZE_PATH = "/Volumes/main/bronze/sales"

df_bronze = (
    df_raw
    .withColumn("_ingestion_ts", current_timestamp())
    .withColumn("_source_file", col("_metadata.file_path"))
)
(
    df_bronze
    .write
    .format("delta")
    .mode("append")
    .option("delta.columnMapping.mode", "name")
    .option("delta.minReaderVersion", "2")
    .option("delta.minWriterVersion", "5")
    .save(BRONZE_PATH)
)

In [0]:
spark.sql("select * from delta.`/Volumes/main/bronze/sales`").show()